In [30]:
# NOTEBOOK IMPORTS
import os, glob, warnings, pickle, re
import numpy as np
from shutil import copyfile, rmtree
from datetime import datetime
from fuzzywuzzy import process

# IMAGE IMPORTS
from PIL import Image

# GIS IMPORTS
from affine import Affine
import pandas as pd

# PLOTTING IMPORTS
import matplotlib.pyplot as plt

# CUSTOM UTILITIES
from IndexUtils import * 

Image.MAX_IMAGE_PIXELS = 933120000
warnings.filterwarnings("ignore")
initialize = False

In [20]:
input_dir = r"C:\Users\fhacesga\Desktop\FIRMsDigitizing\processing\2024-05-12_13-00-40\Inputs\\"
proc_dir  = r"C:\Users\fhacesga\Desktop\FIRMsDigitizing\processing\2024-05-12_13-00-40\Outputs\\"

# LIST ALL IMAGES IN DIRECTORY
image_files = glob.glob(f"{input_dir}/*")

# FILTER IMAGES USING HEURISTICS
patterns = ["IND", "_1."]
index_files = [file for pattern in patterns for file in glob.glob(input_dir + "\\*" + pattern + "*")]
filtered_files = [file for file in image_files if len(os.path.basename(file)) < 12]
index_files.extend(filtered_files)

tiles = list(set(image_files) - set(index_files))
tile_names = [os.path.basename(tile).split(".")[0] for tile in tiles]

with open(f'{proc_dir}IndexCoords.pkl', 'rb') as handle:
    dict = pickle.load(handle)

In [21]:
def filterItems(index_list, elements):
    for e in elements:
        if e in index_list:
            index_list.remove(e)
    return index_list

gen_tile_list = []
gen_bbox_list = []

for k, v in dict.items():
    index_list = list(v.keys())
    index_list = filterItems(index_list, [None, 'county', 'transform_info'])

    coord_list = [v[a]['coords'] for a in index_list]

    gen_tile_list.extend(index_list)
    gen_bbox_list.extend(coord_list)

In [36]:
gen_tile_list = [re.sub("[^0-9]", "", x) for x in gen_tile_list]
tile_names    = [re.sub("[^0-9]", "", x) for x in tile_names]  

In [74]:
for i, p in tqdm(enumerate(tile_names), total=len(tile_names)):
    print(len(p))

  0%|          | 0/1406 [00:00<?, ?it/s]

10
10
8
10
7
8
8
10
9
10
8
7
8
10
10
10
10
10
10
10
10
10
10
10
10
8
9
8
10
10
9
7
8
8
8
10
8
10
7
10
10
7
7
8
10
10
8
10
7
8
8
10
10
9
7
10
7
7
8
7
8
10
9
10
10
9
8
10
8
9
10
10
7
7
8
9
10
7
10
10
9
10
10
10
9
9
10
10
10
7
10
7
8
8
10
10
10
9
10
10
10
10
8
10
10
10
9
7
7
10
10
8
10
10
8
7
9
8
10
7
7
10
7
10
8
7
10
7
10
7
8
9
8
10
10
9
10
10
9
9
7
10
9
10
8
7
10
7
9
10
7
7
8
8
10
8
10
7
8
7
8
8
10
7
10
10
10
10
10
7
8
8
10
7
10
10
10
10
9
10
10
7
9
8
8
10
10
10
10
8
7
9
8
10
10
10
9
7
9
10
10
7
10
8
10
9
10
8
7
8
7
10
8
8
7
7
7
10
10
8
8
7
10
8
7
8
10
10
8
10
8
9
9
10
8
9
9
7
10
7
10
10
8
7
8
9
9
10
10
8
10
8
9
8
10
9
8
8
10
9
9
10
9
10
8
10
10
10
10
10
10
10
7
7
8
10
8
10
10
10
10
7
9
8
10
10
10
8
7
7
10
10
10
9
10
8
7
7
8
7
8
7
7
9
10
10
10
7
8
8
9
10
10
8
8
10
8
8
7
10
10
10
7
9
8
7
9
9
10
10
8
10
10
10
8
10
9
8
10
7
7
7
8
10
9
10
10
10
8
9
10
10
7
9
7
7
10
7
10
8
8
9
8
8
10
7
7
8
8
7
7
8
8
10
9
10
8
7
7
9
9
10
9
8
10
9
10
9
7
7
8
7
10
10
10
8
7
10
9
10
8
8
8
10
7
10
10
8
8
8
7
8
9


In [75]:
stuff = {}

for i, p in tqdm(enumerate(tile_names), total=len(tile_names)):
    name_list = [a[:len(p)] for a in gen_tile_list]
    matches = process.extract(p, name_list)
    print(p, matches)
    out = []
    for t in matches:
        bboxes = []
        if t[1] > 90:
            bboxes.extend([gen_bbox_list[i] for i in range(len(name_list)) if name_list[i] == t[0]])
        stuff[p] = bboxes

  0%|          | 0/1406 [00:00<?, ?it/s]

4854860005 [('485486000', 95), ('4854560005', 90), ('4854690005', 90), ('4854860010', 90), ('4854870005', 90)]
4802330015 [('4802330015', 100), ('4802330005', 90), ('4802330010', 90), ('4802330020', 80), ('4802870015', 80)]
48003563 [('48003506', 88), ('48003506', 88), ('48003506', 88), ('48003503', 88), ('48003503', 88)]
4800450014 [('4800450014', 100), ('4800450014', 100), ('4800450014', 100), ('4800450004', 90), ('4800450042', 90)]
4800356 [('4800350', 86), ('4800350', 86), ('4800350', 86), ('4800350', 86), ('4800350', 86)]
48547040 [('48547001', 88), ('48547002', 88), ('48547000', 88), ('48547000', 88), ('48547000', 88)]
48547010 [('48547001', 88), ('48547002', 88), ('48547000', 88), ('48547000', 88), ('48547000', 88)]
4854700180 [('4854700010', 90), ('4854700120', 90), ('4854700190', 90), ('4854700150', 90), ('4854700210', 90)]
480290478 [('480287047', 78), ('480287048', 78), ('480287048', 78), ('480287047', 78), ('480287047', 78)]
4800450058 [('4800450058', 100), ('4800450056', 9

In [77]:
stuff

{'4854860005': [],
 '4802330015': [],
 '48003563': [],
 '4800450014': [],
 '4800356': [],
 '48547040': [],
 '48547010': [],
 '4854700180': [],
 '480290478': [],
 '4800450058': [],
 '48029615': [],
 '4803013': [],
 '48029661': [],
 '4802670001': [],
 '4802870020': [],
 '4802960120': [[-10598096.250825247,
   3489309.4386451966,
   -10592086.473912403,
   3479179.469982519]],
 '4800350310': [],
 '4800450048': [],
 '4854690035': [],
 '4802870085': [],
 '4800350185': [],
 '4802870230': [],
 '4802870140': [],
 '4854880001': [],
 '4802960015': [],
 '48547038': [],
 '480710180': [],
 '48029651': [],
 '4800450036': [],
 '4802870455': [],
 '482010655': [],
 '4854912': [],
 '48003510': [],
 '48028711': [],
 '48029611': [],
 '4800450024': [],
 '48028765': [],
 '4800390001': [],
 '4800773': [],
 '4854690043': [],
 '4802960170': [],
 '4804243': [],
 '4802672': [],
 '48029663': [],
 '4802960005': [],
 '4802870525': [],
 '48003525': [],
 '4802960145': [],
 '4854704': [],
 '48029658': [],
 '48547048':

In [76]:
i=0
for k, v in stuff.items():
    if len(v) == 0:
        i = i+1
print(f"{i} / {len(stuff)}")

1066 / 1084
